In [ ]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from scipy.optimize import minimize, Bounds
import kaggle_evaluation.default_inference_server as kies

# -----------------------------
# Config / constants
# -----------------------------
DATA_PATH = Path('/kaggle/input/hull-tactical-market-prediction/')
MIN_INVESTMENT = 0.0
MAX_INVESTMENT = 2.0

# -----------------------------
# Utility: evaluation metric
# -----------------------------
class ParticipantVisibleError(Exception):
    pass

def ScoreMetric(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    sol = solution.copy()
    sol['position'] = submission['prediction']

    if sol['position'].max() > MAX_INVESTMENT:
        raise ParticipantVisibleError(f'Position of {sol["position"].max()} exceeds maximum of {MAX_INVESTMENT}')
    if sol['position'].min() < MIN_INVESTMENT:
        raise ParticipantVisibleError(f'Position of {sol["position"].min()} below minimum of {MIN_INVESTMENT}')

    sol['strategy_returns'] = sol['risk_free_rate'] * (1 - sol['position']) + sol['position'] * sol['forward_returns']

    strategy_excess_returns = sol['strategy_returns'] - sol['risk_free_rate']
    strategy_excess_cumulative = (1 + strategy_excess_returns).prod()
    strategy_mean_excess_return = strategy_excess_cumulative ** (1 / len(sol)) - 1
    strategy_std = sol['strategy_returns'].std()
    trading_days_per_yr = 252

    if strategy_std == 0:
        raise ZeroDivisionError

    sharpe = strategy_mean_excess_return / strategy_std * np.sqrt(trading_days_per_yr)
    strategy_volatility = float(strategy_std * np.sqrt(trading_days_per_yr) * 100)

    market_excess_returns = sol['forward_returns'] - sol['risk_free_rate']
    market_excess_cumulative = (1 + market_excess_returns).prod()
    market_mean_excess_return = market_excess_cumulative ** (1 / len(sol)) - 1
    market_std = sol['forward_returns'].std()
    market_volatility = float(market_std * np.sqrt(trading_days_per_yr) * 100)

    excess_vol = max(0, strategy_volatility / market_volatility - 1.2) if market_volatility > 0 else 0
    vol_penalty = 1 + excess_vol

    return_gap = max(0, (market_mean_excess_return - strategy_mean_excess_return) * 100 * trading_days_per_yr)
    return_penalty = 1 + (return_gap ** 2) / 100

    adjusted_sharpe = sharpe / (vol_penalty * return_penalty)
    return min(float(adjusted_sharpe), 1_000_000)

# -----------------------------
# Load train and features
# -----------------------------
train = pd.read_csv(DATA_PATH / "train.csv", index_col="date_id").fillna(0)
main_features = [
    'E1','E10','E11','E12','E13','E14','E15','E16','E17','E18','E19',
    'E2','E20','E3','E4','E5','E6','E7','E8','E9',
    'S2','P9','S1','S5','I2','P8','P10','P12','P13'
]
for c in main_features + ["forward_returns","risk_free_rate"]:
    if c not in train.columns:
        train[c] = 0.0

X_all = train[main_features].values.astype(np.float32)
y_all = train["forward_returns"].values.astype(np.float32).reshape(-1,1)

# -----------------------------
# Model: Updated LSTM Model with More Layers
# -----------------------------
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=3, dropout=0.3):
        super(LSTMModel, self).__init__()
        # Multiple LSTM layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        # Additional fully connected layer after LSTM layers
        self.fc1 = nn.Linear(hidden_size, 128)
        self.fc2 = nn.Linear(128, 1)
    
    def forward(self, x):
        lstm_out, (hn, cn) = self.lstm(x)
        # Pass the LSTM output through additional fully connected layers
        fc_out = torch.relu(self.fc1(lstm_out[:, -1, :]))  # Get output of the last time step
        out = self.fc2(fc_out)
        return out

# -----------------------------
# Data Preprocessing for LSTM
# -----------------------------
def prepare_lstm_data(X, y, seq_len=20):
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_len):
        X_seq.append(X[i:i+seq_len])
        y_seq.append(y[i+seq_len])
    
    X_seq = np.array(X_seq)
    y_seq = np.array(y_seq)
    return X_seq, y_seq

# -----------------------------
# Train LSTM Model
# -----------------------------
def train_lstm(X, y, epochs=100, batch_size=512, lr=1e-3, hidden_size=64, seq_len=20, dropout=0.3):
    X_seq, y_seq = prepare_lstm_data(X, y, seq_len)
    scaler = StandardScaler().fit(X_seq.reshape(-1, X_seq.shape[-1]))
    X_seq_scaled = scaler.transform(X_seq.reshape(-1, X_seq.shape[-1])).reshape(X_seq.shape)
    
    model = LSTMModel(X_seq_scaled.shape[2], hidden_size=hidden_size, dropout=dropout)
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    loss_fn = nn.MSELoss()

    dataset = torch.utils.data.TensorDataset(torch.tensor(X_seq_scaled, dtype=torch.float32), torch.tensor(y_seq, dtype=torch.float32))
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    model.train()
    for ep in range(epochs):
        epoch_loss = 0.0
        for xb, yb in loader:
            opt.zero_grad()
            preds = model(xb)
            loss = loss_fn(preds.squeeze(), yb)
            loss.backward()
            opt.step()
            epoch_loss += loss.item()
        if (ep + 1) % 5 == 0:
            print(f"LSTM epoch {ep + 1}/{epochs} loss={epoch_loss / len(loader):.6f}")
    
    return model, scaler

# -----------------------------
# Train the LSTM model
# -----------------------------
torch.manual_seed(42)
np.random.seed(42)

lstm_model, lstm_scaler = train_lstm(
    X_all, y_all,
    epochs=100,
    batch_size=2048,
    lr=1e-3,
    hidden_size=64,
    seq_len=20,
    dropout=0.2
)

lstm_model.eval()
with torch.no_grad():
    X_seq_scaled = lstm_scaler.transform(X_all.reshape(-1, X_all.shape[1])).reshape(X_all.shape[0], -1, X_all.shape[1])
    lstm_preds_all = lstm_model(torch.tensor(X_seq_scaled, dtype=torch.float32)).numpy().flatten()

mean_lstm = float(np.mean(lstm_preds_all))
lstm_preds_shrunk = 0.8 * lstm_preds_all + 0.2 * mean_lstm

# -----------------------------
# Optimize last 180 days
# -----------------------------
def fun(x):
    sol = train[-180:].copy()
    sub = pd.DataFrame({'prediction': x.clip(0, 2)}, index=sol.index)
    return - ScoreMetric(sol, sub, '')

x0 = np.full(180, 0.05)
res = minimize(fun, x0, method='Powell', bounds=Bounds(lb=0, ub=2), tol=1e-8)
print("Optimizer result:", res.message)
opt_preds = res.x

# -----------------------------
# Tail handling
# -----------------------------
n_total = len(train)
if n_total >= 180:
    lstm_tail = lstm_preds_shrunk[-180:].copy()
else:
    lstm_tail = np.full(180, mean_lstm, dtype=float)

_counter = {"i": 0}

# -----------------------------
# Predict function for inference
# -----------------------------
def predict(test: pl.DataFrame) -> float:
    """Kaggle inference expects a scalar return from this predict."""
    i = _counter["i"]
    idx = min(i, len(opt_preds) - 1)
    opt_p = float(opt_preds[idx])
    lstm_p = float(lstm_tail[idx]) if idx < len(lstm_tail) else mean_lstm
    blended = 0.97 * opt_p + 0.03 * lstm_p
    blended = float(np.clip(blended, 0.0, 2.0))
    _counter["i"] = i + 1
    return blended

# -----------------------------
# Inference server
# -----------------------------
inference_server = kies.DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(('/kaggle/input/hull-tactical-market-prediction/',))

print("Done.")
